In [1]:
import os
import random

import cv2
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import models

from resnet_yolo import resnet50
from yolo_loss import YoloLoss
from dataset import VocDetectorDataset
from eval_voc import evaluate
from predict import predict_image
from config import VOC_CLASSES, COLORS

import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Initialization

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# YOLO network hyperparameters
B = 2  # number of bounding box predictions per cell
S = 14  # width/height of network output grid (larger than 7x7 from paper since we use a different network)

To implement Yolo we will rely on a pretrained classifier as the backbone for our detection network. PyTorch offers a variety of models which are pretrained on ImageNet in the [`torchvision.models`](https://pytorch.org/docs/stable/torchvision/models.html) package. In particular, we will use the ResNet50 architecture as a base for our detector. This is different from the base architecture in the Yolo paper and also results in a different output grid size (14x14 instead of 7x7).

Models are typically pretrained on ImageNet since the dataset is very large (> 1million images) and widely used. The pretrained model provides a very useful weight initialization for our detector, so that the network is able to learn quickly and effictively.

In [4]:
load_network_path = None
pretrained = True

# use to load a previously trained network
if load_network_path is not None:
    print('Loading saved network from {}'.format(load_network_path))
    net = resnet50().to(device)
    net.load_state_dict(torch.load(load_network_path))
else:
    print('Load pre-trained model')
    net = resnet50(pretrained=pretrained).to(device)

Load pre-trained model


In [5]:
learning_rate = 0.001
num_epochs = 50
batch_size = 10

# Yolo loss component coefficients (as given in Yolo v1 paper)
lambda_coord = 5
lambda_noobj = 0.5

In [6]:
criterion = YoloLoss(S, B, lambda_coord, lambda_noobj)
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

## Reading Pascal Data

Since Pascal is a small dataset (5000 in train+val) we have combined the train and val splits to train our detector. This is not typically a good practice, but we will make an exception in this case to be able to get reasonable detection results with a comparatively small object detection dataset.

The train dataset loader also using a variety of data augmentation techniques including random shift, scaling, crop, and flips. Data augmentation is slightly more complicated for detection dataset since the bounding box annotations must be kept consistent through the transformations.

Since the output of the dector network we train is an SxSx(B*5+C), we use an encoder to convert the original bounding box coordinates into relative grid bounding box coordinates corresponding to the the expected output. We also use a decoder which allows us to convert the opposite direction into image coordinate bounding boxes.

In [7]:
file_root_train = 'VOCdevkit_2007/VOC2007/JPEGImages/'
annotation_file_train = 'voc2007.txt'

train_dataset = VocDetectorDataset(root_img_dir=file_root_train,dataset_file=annotation_file_train,train=True, S=S)
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True,num_workers=4)
print('Loaded %d train images' % len(train_dataset))

Initializing dataset
Loaded 5011 train images


In [ ]:
file_root_test = 'VOCdevkit_2007/VOC2007test/JPEGImages/'
annotation_file_test = 'voc2007test.txt'

test_dataset = VocDetectorDataset(root_img_dir=file_root_test,dataset_file=annotation_file_test,train=False, S=S)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False,num_workers=4)
print('Loaded %d test images' % len(test_dataset))

Initializing dataset
Loaded 4950 test images


## Train detector

In [ ]:
best_test_loss = np.inf

for epoch in range(num_epochs):
    net.train()
    
    # Update learning rate late in training
    if epoch == 30 or epoch == 40:
        learning_rate /= 10.0

    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate
    
    print('\n\nStarting epoch %d / %d' % (epoch + 1, num_epochs))
    print('Learning Rate for this epoch: {}'.format(learning_rate))
    
    total_loss = 0.
    
    for i, (images, target) in enumerate(train_loader):
        images, target = images.to(device), target.to(device)
        temp = target.cpu().numpy()
        """
        print("classes", np.max(temp) - np.min(temp))
        print(temp.shape)
        print(np.sum(temp[0,:,:,:10], axis=2))
        print(np.sum(temp[0,:,:,10:], axis=2))
        print(target[0].data.shape)
        print(torch.max(target[0], 2))
        """
        pred = net(images)
        loss = criterion(pred,target)
        total_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 5 == 0:
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f, average_loss: %.4f'
                  % (epoch+1, num_epochs, i+1, len(train_loader), loss.item(), total_loss / (i+1)))
    
    # evaluate the network on the test data
    with torch.no_grad():
        test_loss = 0.0
        net.eval()
        for i, (images, target) in enumerate(test_loader):
            images, target = images.to(device), target.to(device)

            pred = net(images)
            loss = criterion(pred,target)
            test_loss += loss.item()
        test_loss /= len(test_loader)
    
    if best_test_loss > test_loss:
        best_test_loss = test_loss
        print('Updating best test loss: %.5f' % best_test_loss)
        torch.save(net.state_dict(),'best_detector.pth')

    torch.save(net.state_dict(),'detector.pth')
    




Starting epoch 1 / 50
Learning Rate for this epoch: 0.001
629.3767 398.45602 63.458244 7.1409025
624.6666 389.33615 65.059395 6.1604395
527.81866 385.8079 39.862427 2.7968037
545.8825 365.1941 52.581516 6.527246
513.267 347.4227 46.97274 2.9937742
Epoch [1/50], Iter [5/502] Loss: 51.3267, average_loss: 56.8202
448.38965 320.9988 33.695877 1.9788688
467.31055 294.60077 55.92333 4.4525266
398.09265 272.39133 37.60031 3.7591586
367.49997 234.40892 47.073166 4.0061393
387.39883 215.24588 44.96245 2.7652256
Epoch [1/50], Iter [10/502] Loss: 38.7399, average_loss: 49.0970
425.46918 187.20871 70.7391 4.3731318
270.6495 166.7476 30.36647 1.9289246
257.98724 157.22014 31.57911 1.6337279
300.03986 141.14503 49.578304 4.502547
236.46425 122.97431 39.930325 2.7641797
Epoch [1/50], Iter [15/502] Loss: 23.6464, average_loss: 42.6688
230.31888 112.25643 34.486126 2.550928
206.5203 99.89424 43.17329 2.4054794
188.35115 90.45798 35.754936 2.3942792
235.0712 80.932465 59.19713 3.1934242
210.90062 72.0

70.34711 3.282276 28.678587 3.802077
52.659477 3.2905283 22.247921 4.040967
69.46315 3.2564409 24.712574 5.2255096
38.8048 3.25446 13.35865 3.721385
76.234474 3.2292726 28.218044 4.951422
Epoch [1/50], Iter [165/502] Loss: 7.6234, average_loss: 12.7974
88.865105 3.1992097 33.57631 5.321221
47.985123 3.2044768 16.519302 3.1737874
76.5077 3.1820903 26.051123 5.0261025
85.27772 3.1539023 37.628883 7.4943957
65.38213 3.137545 26.72019 5.2979693
Epoch [1/50], Iter [170/502] Loss: 6.5382, average_loss: 12.6352
52.834576 3.136285 19.708584 4.4151187
77.711914 3.1072404 37.112293 4.1184387
57.634827 3.10498 23.893183 3.2501817
89.555565 3.0638273 41.527657 4.376211
48.81774 3.079353 18.998566 4.7875433
Epoch [1/50], Iter [175/502] Loss: 4.8818, average_loss: 12.4608
70.00583 3.043537 23.797554 6.7757316
39.67027 3.053929 16.039206 3.7918396
84.22754 3.0204482 32.59323 4.6563754
83.92368 3.001343 35.6134 5.613982
46.841534 3.0040293 21.866917 4.011083
Epoch [1/50], Iter [180/502] Loss: 4.6842, 

38.897957 1.8241224 17.059452 3.7009132
47.377525 1.816787 20.733875 5.010337
64.04944 1.8036226 27.490131 7.0409417
132.66232 1.7844862 55.922276 7.5235796
54.51717 1.7988002 24.069286 5.4006395
Epoch [1/50], Iter [325/502] Loss: 5.4517, average_loss: 9.5043
47.720825 1.8008188 22.131643 4.508215
90.31655 1.7744956 46.57737 6.472301
84.37657 1.7810813 34.077023 6.674114
39.72968 1.7887803 17.05252 4.6053357
54.74673 1.7765586 24.198666 5.7193418
Epoch [1/50], Iter [330/502] Loss: 5.4747, average_loss: 9.4563
42.018776 1.7801654 17.074976 3.4434834
97.952126 1.7547271 42.627876 7.624423
56.06949 1.7627089 26.103214 4.6253557
55.926647 1.7613434 25.147028 4.3153048
58.99167 1.7532696 22.086212 5.443427
Epoch [1/50], Iter [335/502] Loss: 5.8992, average_loss: 9.4080
59.782166 1.7484291 24.085835 6.0251703
52.00509 1.749831 24.243206 3.968539
76.24306 1.7373494 32.14939 7.6432786
60.447323 1.7317199 32.28074 4.940126
42.921497 1.7349277 20.42433 3.840305
Epoch [1/50], Iter [340/502] Loss:

54.60714 1.2875525 22.435751 6.3805833
46.33284 1.2879344 20.604391 5.3345447
35.365093 1.2896252 16.369669 3.910459
44.54382 1.2859287 20.37199 5.0147433
55.71319 1.2791497 25.205975 4.634681
Epoch [1/50], Iter [485/502] Loss: 5.5713, average_loss: 8.1889
41.581604 1.279476 17.723568 5.702719
36.34681 1.2796482 18.161089 4.5125313
41.497433 1.2740684 16.814114 5.720465
64.17957 1.2692358 28.869936 6.816482
49.50488 1.268241 21.742743 6.7383933
Epoch [1/50], Iter [490/502] Loss: 4.9505, average_loss: 8.1529
75.37624 1.2595198 34.98941 9.058795
62.7654 1.2618859 23.702328 8.385184
55.82135 1.2630451 31.30016 3.8249874
47.23704 1.2615225 17.370975 6.224414
66.41026 1.2528156 24.22232 10.491454
Epoch [1/50], Iter [495/502] Loss: 6.6410, average_loss: 8.1327
36.994675 1.2604197 15.804868 4.571526
64.21709 1.2500224 26.470345 6.185965
48.34996 1.2511445 20.978575 5.3365746
52.263214 1.2495817 21.287846 6.243016
72.45864 1.2410402 35.188957 7.7047796
Epoch [1/50], Iter [500/502] Loss: 7.2459

41.052124 1.2417971 12.404465 8.106319
72.80867 1.233528 36.023743 7.66805
112.78266 1.2238958 47.99084 9.728049
60.842247 1.2368807 26.938799 6.2186794
50.934124 1.2393646 20.992765 6.9121666
59.611607 1.241183 27.857323 6.705892
60.032104 1.2399698 25.118023 8.805143
54.36589 1.2380673 22.203732 7.8013253
38.2901 1.2443429 16.432116 5.4311643
76.707214 1.2347684 36.67276 10.626774
44.411015 1.242438 22.337543 5.536935
47.685276 1.242556 20.932178 5.4041085
45.397408 1.2404932 24.47506 6.0048223
68.94574 1.2354367 29.184212 7.9832726
78.30814 1.2340877 37.196217 3.6637015
54.162262 1.2386482 23.928448 6.0897975
47.05852 1.24124 20.10314 7.976056
64.404884 1.2348285 31.95776 8.354829
63.108406 1.2335103 26.93863 6.869279
60.051086 1.2386674 26.152664 8.016798
68.767845 1.2357323 35.88864 8.5683565
48.58834 1.2418673 22.44647 5.2556977
53.98994 1.2399105 26.399635 6.17853
63.056503 1.2367419 26.000711 8.919545
34.619152 1.2476084 15.005894 5.5854635
40.761696 1.2436998 18.514978 5.90488

...

33.233654 0.5481195 18.758486 9.718059
29.767778 0.63481116 15.737552 8.457726
46.152245 0.5946595 27.391476 11.053957
36.50371 0.5622045 17.486942 10.723525
29.331161 0.58397746 17.437439 6.590091
45.23659 0.8318474 29.040882 10.817791
21.517635 0.6178247 10.573624 8.323205
29.65186 0.6028475 19.499395 6.578735
40.192432 0.5903266 23.308037 11.790984
24.099623 0.5776869 11.317301 7.0955873
32.093906 0.58619636 18.96854 9.394917
33.809265 0.567891 16.30233 10.845786
31.35686 0.67977107 16.917994 10.912661
28.869846 0.57678115 15.600985 5.584934
37.74753 0.6294105 19.863495 13.71372
30.72694 0.71490633 16.996075 8.483663
38.52999 0.633027 20.286009 9.467229
22.715565 0.47085574 11.299208 8.558062
27.450687 0.4970271 16.1805 8.418621
22.961433 0.55422264 8.984389 8.789984
26.136654 0.5698028 12.408582 8.549502
41.690414 0.73604405 23.95403 9.964401
15.952797 0.6080526 7.3904 6.158423
33.569954 0.51220256 17.749199 10.096864
13.08394 0.5550315 5.5134683 6.958517
25.696753 0.5187363 14.323

29.78159 0.49471557 10.779383 12.285716
31.845001 0.6054989 20.082203 8.100642
33.863396 0.6224743 20.853605 7.877509
29.272339 0.6700283 16.09234 9.16134
30.643545 0.8050549 16.742268 5.1739545
32.70679 0.63819927 16.976814 8.144079
24.53678 0.50226396 11.847822 6.666773
35.68984 0.55709 18.788488 8.958795
20.317749 0.5241883 7.5184507 7.0239606
28.1504 0.5723457 14.106131 7.17684
29.755108 0.96596205 17.285046 8.923375
33.73059 0.50698674 14.742344 7.5854373
20.53759 0.53386897 8.885836 8.662363
31.791649 0.74120194 18.296907 8.78701
20.455437 0.5264351 13.374642 5.1220126
23.563053 0.5876433 12.448558 7.863654
50.15178 0.46714008 25.295889 10.001634
46.020794 0.5161883 22.98619 12.216366
43.030148 0.6719235 22.999817 11.343082
37.684353 0.5276292 16.578665 13.987182
23.221825 0.504905 13.06294 7.7093678
34.291172 0.59019995 18.381006 8.133416
29.564621 0.6609757 14.10182 8.300861
25.233793 0.57743293 10.290751 7.7280974
37.003494 0.50091213 21.947813 7.968058
27.28226 0.53723156 14.

35.39505 0.43555593 16.5855 15.961805
17.742172 0.44959968 7.001562 7.605866
33.36424 0.43938372 19.84994 9.840283
21.985966 0.45075482 8.889684 8.034537
24.819832 0.44141945 12.149844 8.5789995
Epoch [18/50], Iter [25/502] Loss: 2.4820, average_loss: 2.7986
36.247715 0.4296488 14.376724 13.593717
14.676221 0.4514852 6.9112883 6.1994743
21.604599 0.44416073 10.027769 8.430599
27.132517 0.43937397 16.597061 7.451628
23.87501 0.45432195 12.942542 7.127791
Epoch [18/50], Iter [30/502] Loss: 2.3875, average_loss: 2.7440
21.08435 0.4508685 7.2597976 11.598244
31.367828 0.44427633 13.0043335 8.751932
22.824615 0.44751722 11.221512 8.323812
22.375244 0.4519792 10.968002 10.211641
30.228977 0.436685 14.57163 8.718934
Epoch [18/50], Iter [35/502] Loss: 3.0229, average_loss: 2.7174
17.107029 0.45113263 7.6446934 7.156038
23.88926 0.43638268 11.492102 7.231099
23.151852 0.45651996 9.50177 10.694318
30.146963 0.45373076 14.869312 8.305058
25.758286 0.4544186 8.70502 9.814244
Epoch [18/50], Iter [4

22.27071 0.58862835 9.78957 8.829697
22.143648 0.55911493 11.08964 5.657324
Epoch [18/50], Iter [180/502] Loss: 2.2144, average_loss: 2.6697
37.47253 0.54742724 20.428923 13.652789
23.820026 0.5700349 14.666966 6.627368
34.765194 0.54810154 21.164549 9.712715
19.7753 0.5685023 9.45223 8.7149515
23.328333 0.55443823 9.585651 10.532994
Epoch [18/50], Iter [185/502] Loss: 2.3328, average_loss: 2.6728
19.965342 0.5726183 8.966943 7.0757804
34.4861 0.5426333 15.632892 13.680319
24.858059 0.5412099 11.4677 9.240138
25.592468 0.56889796 14.267427 8.159815
23.203854 0.5822965 12.256527 8.819629
Epoch [18/50], Iter [190/502] Loss: 2.3204, average_loss: 2.6699
30.149717 0.58752215 16.015871 7.308515
21.670866 0.5661285 10.229386 7.793222
23.024088 0.60591376 10.100409 8.005668
20.55905 0.59807783 8.976504 7.665897
24.65579 0.58274966 12.74598 8.38784
Epoch [18/50], Iter [195/502] Loss: 2.4656, average_loss: 2.6630
21.589905 0.6082926 9.621419 8.220792
19.800232 0.60682905 11.639044 5.275526
19.3

34.133957 0.7321579 16.57597 7.428011
29.526194 0.7530984 12.894001 11.485226
19.01967 0.81676054 11.001211 5.526182
27.745655 0.79350424 12.125887 9.07977
26.4328 0.8525674 14.172125 7.335302
Epoch [18/50], Iter [340/502] Loss: 2.6433, average_loss: 2.6551
32.06415 0.72650224 17.74099 9.553365
38.03396 0.7694717 20.364117 11.397158
22.03424 0.9265559 12.0217905 7.468436
28.679935 0.84825945 16.690273 6.435422
17.2447 0.95733356 8.223367 4.793221
Epoch [18/50], Iter [345/502] Loss: 1.7245, average_loss: 2.6566
21.91024 0.76006377 10.093547 8.286653
26.48368 0.8268306 12.924473 10.876915
28.254326 0.7528145 14.415928 11.166572
27.576366 0.8223544 16.807905 8.629304
20.645031 0.811707 10.643288 6.5197134
Epoch [18/50], Iter [350/502] Loss: 2.0645, average_loss: 2.6543
22.558533 0.8036963 10.349939 6.2931447
26.359985 0.7931856 13.475431 9.407343
20.44204 0.8550182 8.823253 8.02173
32.909496 0.92923856 18.09689 11.959538
22.414642 0.86313087 10.610347 8.409215
Epoch [18/50], Iter [355/502

# View example predictions

In [ ]:
net.eval()

# select random image from test set
image_name = random.choice(test_dataset.fnames)
image = cv2.imread(os.path.join(file_root_test, image_name))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

print('predicting...')
result = predict_image(net, image_name, root_img_directory=file_root_test)
for left_up, right_bottom, class_name, _, prob in result:
    color = COLORS[VOC_CLASSES.index(class_name)]
    cv2.rectangle(image, left_up, right_bottom, color, 2)
    label = class_name + str(round(prob, 2))
    text_size, baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)
    p1 = (left_up[0], left_up[1] - text_size[1])
    cv2.rectangle(image, (p1[0] - 2 // 2, p1[1] - 2 - baseline), (p1[0] + text_size[0], p1[1] + text_size[1]),
                  color, -1)
    cv2.putText(image, label, (p1[0], p1[1] + baseline), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1, 8)

plt.figure(figsize = (15,15))
plt.imshow(image)


## Evaluate on Test

To evaluate detection results we use mAP (mean of average precision over each class)

In [ ]:
evaluate(net, test_dataset_file=annotation_file_test)